In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from astropy.coordinates import SkyCoord
# from astropy.cosmology import FlatLambdaCDM
# from astropy.table import Table
# import GCRCatalogs
# import clmm
# from clmm import ShearAzimuthalAverager



import sys
sys.path.append('../support')
try: import clmm
except:
    import notebook_install
    notebook_install.install_clmm_pipeline(upgrade=False)
    import clmm
from astropy.cosmology import FlatLambdaCDM
import matplotlib.pyplot as plt
import numpy as np
from numpy import random
from sampler import fitters
from astropy.coordinates import SkyCoord
from astropy.table import Table
import GCRCatalogs



Make sure we know which version we're using

In [ ]:
clmm.__version__

In [ ]:
cosmo = FlatLambdaCDM(H0=70, Om0=0.27, Ob0=0.045)

### Read in the extragalactic catalog
- identify the most massive halo
- create a cluster dictionary (ra, dec, z)

In [ ]:
extragalactic_cat = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_small')

In [ ]:
# get list of massive halos in a given redshift and mass range
mmin = 3.e14
mmax = 1.e15
zmin = 0.3
zmax = 0.35

massive_halos = extragalactic_cat.get_quantities(['halo_mass', 'redshift','ra', 'dec'],\
                                                 filters=[f'halo_mass > {mmin}',f'halo_mass < {mmax}','is_central==True',
                                                          f'redshift>{zmin}', f'redshift<{zmax}'])

In [ ]:
select = massive_halos['halo_mass'] == np.max(massive_halos['halo_mass'])

In [ ]:

ra_cl = massive_halos['ra'][select][0]
dec_cl = massive_halos['dec'][select][0]
z_cl = massive_halos['redshift'][select][0]
mass_cl =massive_halos['halo_mass'][select][0]/1.e14

In [ ]:
print (ra_cl, dec_cl, z_cl, mass_cl)

### Apply coordinates and redshift cuts to select backgroud galaxies around the cluster

In [ ]:
# get galaxies around it
ra_min, ra_max = ra_cl-0.3, ra_cl+0.3
dec_min, dec_max = dec_cl-0.3, dec_cl+0.3
z_min = z_cl + 0.3
z_max = 1.5

coord_filters = ['ra >= {}'.format(ra_min),'ra < {}'.format(ra_max),'dec >= {}'.format(dec_min),'dec < {}'.format(dec_max)]
z_filters = ['redshift >= {}'.format(z_min),'redshift < {}'.format(z_max)]

gal_cat = extragalactic_cat.get_quantities(['galaxy_id', 'ra', 'dec', 'shear_1', 'shear_2','redshift', 'convergence'],\
                                           filters=(coord_filters + z_filters))



In [ ]:
#e1, e2 = clmm.utils.convert_shapes_to_epsilon(galcat['shear1'],galcat['shear2'], shape_definition='shear',kappa=galcat['convergence'])

# store the results into an astropy table
# dat = Table([gal_cat['ra'],gal_cat['dec'],e1,\
#       e2,gal_cat['redshift'],gal_cat['galaxy_id']], \
#       names=('ra','dec', 'e1', 'e2', 'z','id'))

dat = Table([gal_cat['ra'],gal_cat['dec'],gal_cat['shear_1'],\
      gal_cat['shear_2'],gal_cat['redshift'],gal_cat['galaxy_id']], \
      names=('ra','dec', 'e1', 'e2', 'z','id'))

cl = clmm.GalaxyCluster('CL', ra_cl, dec_cl, z_cl, dat)   

### Computes the tangential and cross shear profiles

#### Default binning

In [ ]:
bin_edges = clmm.polaraveraging.make_bins(0.1, 4, 15, method='evenlog10width')
cl.compute_shear(geometry="flat")
profile = cl.make_shear_profile("radians", "Mpc", bins=bin_edges,cosmo=cosmo)

In [ ]:
plt.errorbar(profile['radius'],profile['gt'],yerr=profile['gt_err'])
plt.xscale('log')
plt.yscale('log')
plt.xlabel('R (Mpc)')
plt.ylabel('reduced tangential shear')

### Set up a model

In [ ]:
reduced_shear_model = np.zeros(len(profile['radius']))

for i,r in enumerate(profile['radius']): 
    reduced_shear = clmm.predict_reduced_tangential_shear(r*cosmo.h,mass_cl*cosmo.h, 4.,z_cl, profile['z'], cosmo,
                                                        delta_mdef=200,halo_profile_model='nfw')  
    reduced_shear_model[i] = np.nanmean(reduced_shear)    

In [ ]:
plt.errorbar(profile['radius'],profile['gt'],yerr=profile['gt_err'])
plt.plot(profile['radius'],reduced_shear_model, label='model shear')
plt.legend()
plt.xscale('log')
plt.yscale('log')
plt.xlabel('R (Mpc)')
plt.ylabel('<shear>')